In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from loguru import logger
import argparse


class PickleWrapper(object):

    @classmethod
    def loadFromFile(cls, file, mode='rb'):
        with open(file, mode) as f:
            return pickle.load(f)

    @classmethod
    def dump2File(cls, o, file, mode='wb'):
        '''
        把目标对象序列化到文件
        :param o: 目标对象
        :param file: 文件
        :param mode:
        :return:
        '''
        with open(file, mode) as f:
            pickle.dump(o, f)



parser = argparse.ArgumentParser(description='t')
parser.add_argument('--now_phase', type=int, default=6, help='')
parser.add_argument('--window', type=int, default=10, help='cocur_matr的时间窗')
parser.add_argument('--time_decay', type=float, default=7/8, help='时间衰减')
parser.add_argument('--mode', type=str, default='train', help='train test')
parser.add_argument('--topk', type=int, default=500, help='每种召回策略召回的样本数')
parser.add_argument('--DATA_DIR', type=str, default='./', help='data dir')

args = parser.parse_args(args=[])
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))

# Cell
def load_click_data_per_phase(now_phase, base_dir):
    """
    """


    all_click_df = []
    for c in range(now_phase + 1):
        logger.info(f'phase: {c}')
        cols_str = 'user_id item_id time'.split()
        click_train1 = pd.read_csv( './underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'], converters={c: str for c in cols_str})
        click_test1 = pd.read_csv( './underexpose_test_click-{}.csv'.format(c, c), header=None,  names=['user_id', 'item_id', 'time'], converters={c: str for c in cols_str})
        test_qtime1 = pd.read_csv( './underexpose_test_qtime-{}.csv'.format(c, c), header=None,  names=['user_id','time'], converters={c: str for c in cols_str})
        click_test1_val = click_test1.sort_values(['user_id', 'time']).drop_duplicates(subset=['user_id'],keep='last')

        click_test1 = click_test1[~click_test1.index.isin(click_test1_val.index)]
        all_click = click_train1.append(click_test1).drop_duplicates().sort_values('time')

        all_click_df.append((all_click, click_test1_val, test_qtime1))
        logger.info(f'all_click: {all_click.shape}, click_test1_val: {click_test1_val.shape}, test_qtime1: {test_qtime1.shape}')
    return all_click_df
all_click_df = load_click_data_per_phase(args.now_phase, args.DATA_DIR)
all_train=pd.concat([all_click_df[i][0] for i in range(0,7)]).drop_duplicates().reset_index(drop=True)
all_train=all_train.sort_values(by=['user_id','time'],ascending=['True','False'])

import multiprocessing

item2vec=all_train.groupby('user_id')['item_id'].agg(list)
item2vec
# user2vec=all_train.groupby('item_id')['user_id'].agg(list)
# user2vec

# sentences=list(user2vec.values)
sentences=list(item2vec.values)



import gensim
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''callback to print loss after each epoch'''
    def __init__(self):
        self.epoch=0
        self.last_loss=0
    def on_epoch_end(self,model):
        loss=model.get_latest_training_loss()
        now_loss=loss-self.last_loss
        self.last_loss=loss
        test_word=list(model.wv.vocab.keys())[0]
        print('Loss after epoch {}: {}'.format(self.epoch,now_loss))
        print('for word {}: the array now is: {}'.format(test_word, model.wv.vectors[0][:10]))              
        print('for word {}: the top 3 sim user is {}'.format(test_word,model.wv.most_similar(test_word,topn=3)))                                                
        self.epoch+=1

# model = gensim.models.Word2Vec(sentences,
#                     size=256,
#                     alpha=0.02,
#                     window=999999,
#                     min_count=1,
#                     workers=multiprocessing.cpu_count(),
#                     compute_loss=True,
#                     iter=35,
#                     hs=0,
#                     sg=1,
#                     seed=42,callbacks=[callback()])
# # model.save('../../user_data/user2vec-0.4')

model=gensim.models.Word2Vec.load('./item2vec-0.4a')

iv_df=pd.DataFrame(model.wv.vectors,index=model.wv.vocab.keys(),columns=['iv'+str(i) for i in range(0,128)]).reset_index()

iv_df.shape

import sys

print(sys.version, sys.platform, sys.executable)
from tqdm import tqdm

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.precision',16)

train_path = './'
test_path = './'


col_name = ['item_id']
for i in range(0, 128):
    col_name.append('tv' + str(i))
for i in range(0, 128):
    col_name.append('iv' + str(i))

itemft = pd.read_csv(train_path + 'underexpose_item_feat.csv', low_memory=False, names=col_name)
itemft = itemft.replace('[\[\]]', '', regex=True)  # regex=True is the key


itemft.shape
iv_df.shape


iv_df['index']=iv_df['index'].astype(int)

itemft=itemft.merge(iv_df,left_on='item_id',right_on='index',how='left')
itemft.shape



itemft.drop(['index'],axis=1,inplace=True)

itemft.fillna(0,inplace=True)

itemft.dropna().shape

vec = itemft.iloc[:, 1:].to_numpy()
vec.shape




chunk_size = 500
matrix_len = vec.shape[0]


def sim_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(vec[start:end], vec)


import numpy as np

sim_matrix = np.ndarray(shape=(matrix_len, matrix_len))

# ---------------------------------------------------------------------

for start in tqdm(range(0, matrix_len, chunk_size)):
    cos_sim_chunk = sim_chunk(start, start + chunk_size)
    if (start + chunk_size) <= matrix_len:
        sim_matrix[start:start + chunk_size] = cos_sim_chunk
    else:
        sim_matrix[start:matrix_len] = cos_sim_chunk


results = {}

recall_num=501
for idx, row in tqdm(itemft[['item_id']].iterrows(),total=sim_matrix.shape[0]):
    similar_indices = sim_matrix[idx].argsort()[
                      :-recall_num:-1]  # ascending return index, : means reverse order, -100:-1 means last 100-1 to last 1-1 position
    similar_items = [(sim_matrix[idx][i], itemft['item_id'][i]) for i in similar_indices]

    results[row['item_id']] = similar_items[1:]
    



for p in tqdm(range(0,6+1)):
    user_train=all_click_df[p][0].sort_values(by=['user_id','time'],ascending=[True,False])
    user_true=all_click_df[p][1] 
    recall_num=500
    itemlist=list(itemft.item_id.unique())
    recall_frame = pd.DataFrame(columns=['user_id', 'item_id_pred', 'score', 'rank', 'item_id_true'],index=range(0,recall_num*len(user_train.user_id.unique())))
    row_num=0
    for user in tqdm(user_true.user_id.unique()):
        latest_item = user_train.loc[user_train.user_id == user, 'item_id'].values[0]
        true_item = user_true.loc[user_true.user_id == user, 'item_id'].values[0]
    #     history_item=user_train.groupby(['user_id'])['item_id'].agg(list)[user]
        if int(latest_item) in itemlist: 
            recs = results[int(latest_item)][:recall_num]
            rank=1
            for i in range(0, recall_num-1):
    #             if str(recs[i][1]) not in history_item:
    #                 recall_frame.iloc[row_num]['user_id'] = user
    #                 recall_frame.iloc[row_num]['item_id_pred'] = recs[i][1]
    #                 recall_frame.iloc[row_num]['score'] = recs[i][0]
    #                 recall_frame.iloc[row_num]['rank'] = rank
    #                 rank=rank+1
    #                 recall_frame.iloc[row_num]['item_id_true'] = true_item
    #                 row_num+=1
                recall_frame.iloc[row_num]['user_id'] = user
                recall_frame.iloc[row_num]['item_id_pred'] = recs[i][1]
                recall_frame.iloc[row_num]['score'] = recs[i][0]
                recall_frame.iloc[row_num]['rank'] = rank
                rank=rank+1
                recall_frame.iloc[row_num]['item_id_true'] = true_item
                row_num+=1
        else:
    #         print(user,latest_item,latest_item in itemlist)
            recall_frame.iloc[row_num]['user_id'] = user
            recall_frame.iloc[row_num]['item_id_pred'] = np.nan
            recall_frame.iloc[row_num]['score'] = np.nan
            recall_frame.iloc[row_num]['rank'] = np.nan
            recall_frame.iloc[row_num]['item_id_true'] = true_item
            row_num+=1
    recall_frame=recall_frame.dropna(how='all').astype('float32')
    print(sum(recall_frame.item_id_pred == recall_frame.item_id_true)/1821)
    recall_frame.dropna().to_csv('128vec-add-i2i.csv')

_StoreAction(option_strings=['--now_phase'], dest='now_phase', nargs=None, const=None, default=6, type=<class 'int'>, choices=None, help='', metavar=None)

_StoreAction(option_strings=['--window'], dest='window', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='cocur_matr的时间窗', metavar=None)

_StoreAction(option_strings=['--time_decay'], dest='time_decay', nargs=None, const=None, default=0.875, type=<class 'float'>, choices=None, help='时间衰减', metavar=None)

_StoreAction(option_strings=['--mode'], dest='mode', nargs=None, const=None, default='train', type=<class 'str'>, choices=None, help='train test', metavar=None)

_StoreAction(option_strings=['--topk'], dest='topk', nargs=None, const=None, default=500, type=<class 'int'>, choices=None, help='每种召回策略召回的样本数', metavar=None)

...

(98769, 129)

(108916, 386)

(108916, 385)

(108916, 384)

 10%|▉         | 10778/108916 [03:37<29:09, 56.08it/s]  